In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import pyproj
import json

In [ ]:
def create_grid_cell(x_center, y_center, width=50, height=50):
    half_width = width / 2
    half_height = height / 2
    return Polygon([
        (x_center - half_width, y_center - half_height),
        (x_center - half_width, y_center + half_height),
        (x_center + half_width, y_center + half_height),
        (x_center + half_width, y_center - half_height)
    ])


In [ ]:
data = pd.read_csv("./2023_pollutant_grid_avg.csv")
data['geometry'] = data.apply(lambda row: create_grid_cell(row['x'], row['y']), axis=1)

In [ ]:
# Drop the x_center and y_center columns
data = data.drop(columns=['x', 'y'])

In [ ]:
gdf = gpd.GeoDataFrame(data, crs="EPSG:25833")
gdf = gdf.to_crs("EPSG:4326")
gdf.to_file("out/all.geojson", driver="GeoJSON")

In [ ]:
# Function to merge grid cells and save to a file
def merge_and_save(gdf, pollutant, filename):
    gdf_merged = gdf.dissolve(by=pollutant, as_index=False)  # Merge by pollutant value
    gdf_merged = gdf_merged[[pollutant, 'geometry']]  # Keep only pollutant and geometry
    gdf_merged.to_file(filename, driver="GeoJSON")
    print(f"Saved: {filename}")

In [ ]:
# Merge and save files for NO₂, PM10, and PM2.5
merge_and_save(gdf, 'no2', 'out/merged_no2.geojson')
merge_and_save(gdf, 'pm10', 'out/merged_pm10.geojson')
merge_and_save(gdf, 'pm2.5', 'out/merged_pm25.geojson')

In [ ]:
# Calculate min and max values for NO₂, PM10, and PM2.5
pollutants = ["no2", "pm10", "pm2.5"]
pollutant_stats = {
    pollutant: [int(data[pollutant].min()),int(data[pollutant].max()) ]
    for pollutant in pollutants
}

# Write to a JSON file
output_filename = "../src/lib/pollutant_stats.json"
with open(output_filename, "w") as json_file:
    json.dump(pollutant_stats, json_file, indent=4)

print(f"Pollutant statistics saved to {pollutant_stats}")

In [ ]:
#
!tippecanoe --output-to-directory ../public/tiles/highzoom --no-tile-compression --force -B 13 --minimum-zoom=10 --maximum-zoom=13 ./out/all.geojson

In [ ]:
!tippecanoe --output-to-directory ../public/tiles/merged_no2 --no-tile-compression --force --minimum-zoom=1 --maximum-zoom=10 --no-tile-size-limit ./out/merged_no2.geojson

In [ ]:
!tippecanoe --output-to-directory ../public/tiles/merged_pm10 --no-tile-compression --force -B 13 --minimum-zoom=1 --maximum-zoom=10 --no-tile-size-limit ./out/merged_pm10.geojson

In [ ]:
!tippecanoe --output-to-directory ../public/tiles/merged_pm25 --no-tile-compression --force -B 13 --minimum-zoom=1 --maximum-zoom=10 --no-tile-size-limit ./out/merged_pm25.geojson